<a href="https://colab.research.google.com/github/it21250156/MyFirstMLProject/blob/main/SalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
%pip install -Uq upgini catboost

In [26]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=10_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()



,date,store,item,sales
0,2013-01-01,3,12,38
1,2013-01-01,4,9,19
2,2013-01-01,10,21,33
3,2013-01-01,3,27,11
4,2013-01-01,2,3,19


In [27]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [28]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [29]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    CV = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=edb3cf8d-7580-4f38-8743-349517530660
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


86 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_31_fa6d9a99,0.0287,100.0000,"5.1172, 5.0879, 4.9169",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_pca_0_d7e0a1fc,0.0263,100.0000,"21.2914, 28.5398, 1.2874",Upgini,Weather & climate normals data,Daily
f_autofe_div_89c56a5f,0.0234,100.0000,"0.6895, 1.1854, -1.2125",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_weather_date_weather_umap_48_b39cd0c4,0.0200,100.0000,"5.4001, 5.4695, 5.8517",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_34_c3ef5b4f,0.0185,100.0000,"5.6284, 5.5371, 5.1219",Upgini,Weather & climate normals data,Daily
f_autofe_div_d6db5d7a,0.0176,100.0000,"1.3001, 0.8399, 1.5568",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_d0993cdc,0.0143,100.0000,"354.6602, 348.2182, 305.198",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_08e89348,0.0084,100.0000,"79.7881, 19.2991, 38.6147",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_0a5adf97,0.0069,100.0000,"-0.0147, 0.1182, -0.2472",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_c256dd85,0.0068,100.0000,"1.0084, 0.5932, -0.5012",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_b97cf188,0.0064,85.3155,"-0.9241, -0.9241, -1.437",Upgini,AutoFE: features from Calendar data,Daily


Upgini,Weather & climate normals data,0.1014,14
Upgini,"AutoFE: features from Calendar data,Markets data",0.0978,39
Upgini,AutoFE: features from Markets data,0.0189,15
Upgini,AutoFE: features from Calendar data,0.0138,15
Upgini,Calendar data,0.0053,2
Upgini,World economic indicators,0.0009,1


"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_div_d6db5d7a,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,/
Markets data,f_autofe_div_d0993cdc,f_financial_date_stoxx_043cbcd4,f_financial_date_stoxx_7d_to_1y_2ea815f4,/
"Calendar data,Markets data",f_autofe_mul_08e89348,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_div_0a5adf97,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_92dac942,/
"Calendar data,Markets data",f_autofe_mul_c256dd85,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,*
Calendar data,f_autofe_div_b97cf188,f_events_date_week_cos3_7525fe31,f_events_date_week_sin2_e28440a5,/
"Calendar data,Markets data",f_autofe_div_1edd5009,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,/
"Calendar data,Markets data",f_autofe_mul_6a97c336,f_events_date_week_cos1_f6a8c1fc,f_financial_date_vix_7d_to_1y_634c77eb,*
"Calendar data,Markets data",f_autofe_mul_0cc09d2d,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_mul_42dfbbc4,f_events_date_week_sin1_847b5db1,f_financial_date_silver_7d_to_1y_0ccfe462,*



Examples of outliers with maximum value of target:
40    205
24    196
46    176
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,7988,50.1955,281.0876,160.8151,120.2726
Eval 1,2012,59.4155,386.7405,239.1702,147.5702


In [30]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,7988,50.1955,0.299435,0.159276,0.140159
1,Eval 1,2012,59.4155,0.251622,0.146766,0.104856


In [31]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=fb408e70-70d6-4af6-bb7e-fbd501e2608e
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=ecf1e5af-2c52-4599-a3dd-eaec5210475d
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_weather_date_weather_umap_31_fa6d9a99,f_weather_date_weather_pca_0_d7e0a1fc,f_autofe_div_89c56a5f,f_weather_date_weather_umap_48_b39cd0c4,f_weather_date_weather_umap_34_c3ef5b4f,f_autofe_div_d6db5d7a,f_autofe_div_d0993cdc,...,f_autofe_mul_65b532e8,f_autofe_mul_6b3213a9,f_autofe_mul_89a7ef9a,f_autofe_mul_9a2dc29a,f_autofe_mul_9a4ab68c,f_autofe_mul_ecb39f07,f_events_date_year_sin2_59955ffd,f_weather_date_weather_umap_16_25ac8b87,f_weather_date_weather_umap_28_b90870a1,f_weather_date_weather_umap_47_5123ef0a
0,2013-01-01,3,12,4.712653,29.676683,0.880047,4.540985,5.664261,1.023385,261.634582,...,19.200886,0.206448,0.486478,0.975557,18.812558,0.870528,0.337523,5.140265,4.934963,5.927147
1,2013-01-01,4,9,4.712653,29.676683,0.880047,4.540985,5.664261,1.023385,261.634582,...,19.200886,0.206448,0.486478,0.975557,18.812558,0.870528,0.337523,5.140265,4.934963,5.927147
2,2013-01-01,10,21,4.712653,29.676683,0.880047,4.540985,5.664261,1.023385,261.634582,...,19.200886,0.206448,0.486478,0.975557,18.812558,0.870528,0.337523,5.140265,4.934963,5.927147
3,2013-01-01,3,27,4.712653,29.676683,0.880047,4.540985,5.664261,1.023385,261.634582,...,19.200886,0.206448,0.486478,0.975557,18.812558,0.870528,0.337523,5.140265,4.934963,5.927147
4,2013-01-01,2,3,4.712653,29.676683,0.880047,4.540985,5.664261,1.023385,261.634582,...,19.200886,0.206448,0.486478,0.975557,18.812558,0.870528,0.337523,5.140265,4.934963,5.927147


In [32]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[39.56447289695444]

In [33]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

[16.28611245815716]